# Data Alignmnent 
## normalize dates

In [12]:
import pandas as pd

# Load news dataset (analyst ratings)
news_df = pd.read_csv('../assets/data/raw_analyst_ratings.csv')

# Load stock price datasets for each stock
stock_dfs = {
    'AAPL': pd.read_csv('../assets/data/AAPL_historical_data.csv'),
    'AMZN': pd.read_csv('../assets/data/AMZN_historical_data.csv'),
    'GOOG': pd.read_csv('../assets/data/GOOG_historical_data.csv'),
    'MSFT': pd.read_csv('../assets/data/MSFT_historical_data.csv'),
    'META': pd.read_csv('../assets/data/META_historical_data.csv'),
    'NVDA': pd.read_csv('../assets/data/NVDA_historical_data.csv'),
    'TSLA': pd.read_csv('../assets/data/TSLA_historical_data.csv')
}

# Normalize timestamps for news data (ensure both date and time are captured)
news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce')

# Drop rows with NaT in the 'date' column
news_df = news_df.dropna(subset=['date'])

# Normalize timestamps for stock price data (ensure both date and time are captured)
for stock, stock_data in stock_dfs.items():
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], errors='coerce')

# Convert stock Date columns to timezone-aware (UTC) to match news data
for stock, stock_data in stock_dfs.items():
    if stock_data['Date'].dt.tz is None:  # Check if timezone is naive
        stock_data['Date'] = stock_data['Date'].dt.tz_localize('UTC')  # Localize to UTC
    else:
        stock_data['Date'] = stock_data['Date'].dt.tz_convert('UTC')  # Convert to UTC if already aware

# Convert news date column to UTC timezone (if it's not already)
if news_df['date'].dt.tz is None:  # Check if timezone is naive
    news_df['date'] = news_df['date'].dt.tz_localize('UTC')  # Localize to UTC
else:
    news_df['date'] = news_df['date'].dt.tz_convert('UTC')  # Convert to UTC if already aware

# Align news with stock prices
aligned_dfs = {}

for stock, stock_data in stock_dfs.items():
    # Merge the stock data with news data based on the nearest date
    aligned_news = pd.merge_asof(
        stock_data.sort_values('Date'),
        news_df.sort_values('date'),
        left_on='Date',
        right_on='date',
        direction='nearest'
    )
    aligned_dfs[stock] = aligned_news

# Handle missing data by dropping rows with missing values after the merge
for stock in aligned_dfs:
    aligned_dfs[stock] = aligned_dfs[stock].dropna()

# Example: Check the first few rows of the aligned data for AAPL
aapl_aligned_data = aligned_dfs['AAPL']
print(aapl_aligned_data.head())


                       Date      Open      High       Low     Close  \
0 1980-12-12 00:00:00+00:00  0.128348  0.128906  0.128348  0.128348   
1 1980-12-15 00:00:00+00:00  0.122210  0.122210  0.121652  0.121652   
2 1980-12-16 00:00:00+00:00  0.113281  0.113281  0.112723  0.112723   
3 1980-12-17 00:00:00+00:00  0.115513  0.116071  0.115513  0.115513   
4 1980-12-18 00:00:00+00:00  0.118862  0.119420  0.118862  0.118862   

   Adj Close     Volume  Dividends  Stock Splits  Unnamed: 0  \
0   0.098943  469033600        0.0           0.0      357064   
1   0.093781  175884800        0.0           0.0      357064   
2   0.086898  105728000        0.0           0.0      357064   
3   0.089049   86441600        0.0           0.0      357064   
4   0.091630   73449600        0.0           0.0      357064   

                                            headline  \
0  ETFs To Watch April 28, 2011 (DGP, IEO, PRN, IDX)   
1  ETFs To Watch April 28, 2011 (DGP, IEO, PRN, IDX)   
2  ETFs To Watch Apr

# Sentiment Analysis:

In [14]:
from textblob import TextBlob

# Load data from a CSV file
file_path = "../assets/data/raw_analyst_ratings.csv"  # Replace with the actual path to your file
df = pd.read_csv(file_path)

# Ensure the dataset has a 'headline' column
if 'headline' not in df.columns:
    raise ValueError("The dataset must contain a 'headline' column.")

# Perform sentiment analysis
def analyze_sentiment(headline):
    analysis = TextBlob(headline)
    polarity = analysis.polarity  # Sentiment polarity: -1 (negative) to +1 (positive)
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    else:
        return "Neutral"

# Apply the sentiment analysis function to the headlines
df['sentiment'] = df['headline'].apply(analyze_sentiment)

# Count the number of headlines by sentiment
sentiment_counts = df['sentiment'].value_counts()

# Print results
print("Sentiment Analysis Results:")
print(sentiment_counts)


Sentiment Analysis Results:
sentiment
Neutral     934914
Positive    341178
Negative    131236
Name: count, dtype: int64


# Daily Stock Returns: 

In [15]:
import pandas as pd

# List of stock filenames
stock_files = {
    'AAPL': '../assets/data/AAPL_historical_data.csv',
    'AMZN': '../assets/data/AMZN_historical_data.csv',
    'GOOG': '../assets/data/GOOG_historical_data.csv',
    'MSFT': '../assets/data/MSFT_historical_data.csv',
    'META': '../assets/data/META_historical_data.csv',
    'NVDA': '../assets/data/NVDA_historical_data.csv',
    'TSLA': '../assets/data/TSLA_historical_data.csv'
}

# Dictionary to store daily returns for each stock
daily_returns = {}

# Loop through each stock and calculate the daily returns
for stock, file_path in stock_files.items():
    # Load stock data
    stock_data = pd.read_csv(file_path)

    # Ensure the 'Date' column is in datetime format
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])

    # Set 'Date' as the index
    stock_data.set_index('Date', inplace=True)

    # Calculate daily returns based on the 'Close' price (percentage change)
    stock_data['Daily_Return'] = stock_data['Close'].pct_change() * 100

    # Store the daily returns in the dictionary
    daily_returns[stock] = stock_data[['Close', 'Daily_Return']]

    # Optionally, save the results to a new CSV file
    stock_data.to_csv(f'../assets/data/{stock}_with_daily_returns.csv')

# Example: Print the first few rows of the daily returns for AAPL
print(daily_returns['AAPL'].head())


               Close  Daily_Return
Date                              
1980-12-12  0.128348           NaN
1980-12-15  0.121652     -5.217061
1980-12-16  0.112723     -7.339788
1980-12-17  0.115513      2.475091
1980-12-18  0.118862      2.899246


# Aggregate Sentiments:

In [24]:
import pandas as pd
from textblob import TextBlob

# Sample news data with 'date' and 'sentiment_score' columns
news_data = pd.read_csv('../assets/data/raw_analyst_ratings.csv')

# Ensure the 'date' column is in datetime format
news_data['date'] = pd.to_datetime(news_data['date'], errors='coerce')

print(news_data.columns) 

# Function to calculate sentiment polarity using TextBlob
def get_sentiment(text):
    blob = TextBlob(str(text))  # Ensure text is in string format
    return blob.sentiment.polarity  # You can also use blob.sentiment.subjectivity if needed

# Calculate sentiment scores for each article
news_data['sentiment_score'] = news_data['headline'].apply(get_sentiment)

# Check if timezone is naive and localize or convert to UTC
if news_data['date'].dt.tz is None:  # Check if timezone is naive
    news_data['date'] = news_data['date'].dt.tz_localize('UTC')  # Localize to UTC
else:
    news_data['date'] = news_data['date'].dt.tz_convert('UTC')

# Calculate daily average sentiment scores by grouping by 'date'
daily_sentiment = news_data.groupby('date')['sentiment_score'].mean().reset_index()

# Display the first few rows of the aggregated sentiment scores
print(daily_sentiment.head())

# Optionally, save the aggregated data to a CSV file
daily_sentiment.to_csv('../assets/data/daily_sentiment_scores.csv', index=False)


Index(['Unnamed: 0', 'headline', 'url', 'publisher', 'date', 'stock'], dtype='object')
                       date  sentiment_score
0 2011-04-28 01:01:48+00:00         0.000000
1 2011-04-28 17:49:29+00:00         0.136364
2 2011-04-28 19:00:36+00:00         0.000000
3 2011-04-29 17:47:06+00:00        -0.166667
4 2011-04-29 20:11:05+00:00         0.500000


# Correlation